We start with basic imports to use blablacar web api.

In [1]:
#%pip install pandas

In [2]:
#%pip install fsspec

In [3]:
import json
import requests
import pandas as pd

Dès qu'on reçoit l'API on part là-dessus : https://dev.blablacar.com/hc/en-us/articles/360012064820--Search-V3-API-Documentation

In [4]:
api_token = open("./api/api.txt","r").read()
api_url = "https://public-api.blablacar.com"

fn=Rennes%2C%20France&fc=48.117266%2C-1.677793&
tn=Brest%2C%20France&tc=48.390394%2C-4.486076&
tcc=FR&formatted_to=Brest&db=2020-08-21&
departure_location_id=eyJpIjoiQ2hJSmhaRFdweV9lRGtnUk1LdmtOczJsREFRIiwicCI6MSwidiI6MSwidCI6W119&
arrival_location_id=eyJpIjoiQ2hJSmsxdVMyZUc3RmtnUnF6Q2NGMWlEU01ZIiwicCI6MSwidiI6MSwidCI6W119&
departure_city=Rennes&
arrival_city=Brest&seats=1&searchOrigin=search_edit
    

In [5]:
from_coordinate = "48.117266%2C-1.677793" # string : latitude, longitude - RENNES
to_coordinate = "48.390394%2C-4.486076" # string : latitude, longitude - BREST
from_country = "FR" # string : ISO_3166_1
to_country = "FR" # string : ISO_3166_1
locale = "fr-FR" # string : locale in which the result will be displayed https://dev.blablacar.com/hc/en-us/articles/360008930020
currency = "EUR" # string : ISO_4217 currency
number_of_trips_we_want_returned = "100" # int : from 1 to 100, default is 10 
start_date_local = "2020-08-21T17:00:00" # string(date-time) : Format: "yyyy-M-dTHH:mm:ss"
requested_seats = "1" # int : minimum number of seats requested, default value is 1
radius_in_meters = "6000" # int : radius of the circle around the coordinates in which trips will be searched from (in meters)

In [6]:
response = requests.get(api_url+"/api/v3/trips?key="+api_token+"&from_coordinate="+from_coordinate+"&from_country="
                        +from_country+"&to_coordinate="+to_coordinate+"&to_country="+to_country+"&locale="+locale+"&currency="
                        +currency+"&count="+number_of_trips_we_want_returned+"&start_date_local="+start_date_local
                        +"&radius_in_meters="+radius_in_meters+"&sort=price:asc") 

In [28]:
print(response.status_code) # SHOULD BE 200

200


In [12]:
json_trips = response.json() # it's a dictionnary

In [14]:
for json_ in json_trips['trips']:
    print('\n')
    print(json_)
    print('\n')
    print('--------------------------------------------------------------------------------------------------')



{'link': 'https://www.blablacar.fr/trip?source=CARPOOLING&id=2030433816-rennes-brest', 'waypoints': [{'date_time': '2020-08-21T17:00:00Z', 'place': {'city': 'Rennes', 'address': 'Campus Beaulieu, Rennes', 'latitude': 48.1218, 'longitude': -1.64354, 'country_code': 'FR'}}, {'date_time': '2020-08-21T20:20:00Z', 'place': {'city': 'Brest', 'address': 'Fort Montbarey, Brest', 'latitude': 48.3786, 'longitude': -4.54985, 'country_code': 'FR'}}], 'price': {'amount': '11.25', 'currency': 'EUR'}, 'vehicle': {'make': 'PEUGEOT', 'model': '208'}, 'distance_in_meters': 255543, 'duration_in_seconds': 12000}


--------------------------------------------------------------------------------------------------


{'link': 'https://www.blablacar.fr/trip?source=CARPOOLING&id=2026019926-saint-jacques-de-la-lande-brest', 'waypoints': [{'date_time': '2020-08-21T18:00:00Z', 'place': {'city': 'Saint-Jacques-de-la-Lande', 'address': 'castorama rennes st jacques', 'latitude': 48.08798, 'longitude': -1.704612, 'c

In [42]:
new_dict = {'trip_id':[],'price_of_the_trip':[]}
for dict_trip in json_trips["trips"]:
    trip_id =  dict_trip["link"].split("id=")[1].split("-")[0]
    #number_of_passenger =  dict_trip["link"].split("id=")[1].split("-")[0]
    price_of_the_trip = dict_trip["price"]["amount"]
    new_dict['trip_id'].append(trip_id)
    new_dict['price_of_the_trip'].append(price_of_the_trip)

In [43]:
# text_trips = response.text # it's a text

In [44]:
df_trips = pd.DataFrame.from_dict(new_dict)
df_trips.head()

,trip_id,price_of_the_trip
0,2030433816,11.25
1,2026019926,12.50
2,2014886656,12.50
3,2030108726,12.50
4,2029796031,13.75
